In [37]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
from collections import defaultdict
import random
import cv2

In [38]:
transform = transforms.Compose([
    transforms.ToTensor(),
])

In [39]:
# for training need train and test set
Traindataset = torchvision.datasets.CIFAR10(root='./dataset', train=True, download=True, transform=transform)
Testdataset =  torchvision.datasets.CIFAR10(root='./dataset', train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [40]:
# Need to have complete dataset for CNN later
CompleteData = torch.utils.data.ConcatDataset([Traindataset, Testdataset])

In [41]:
# order of the CIFAR10 found online
class_labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
ClassSelected = ['airplane', 'horse', 'bird', 'cat', 'deer']
Class_data = defaultdict(list)
for i in CompleteData:
    # check overall image size to see if all class have 100 data, if yes break out the loop
    if all(len(Class_data[count]) >= 100 for count in ClassSelected):
        break
    else:
        image, labelName = i
        className = class_labels[labelName]
        if className in ClassSelected:
            # do not append if a single class has more than 100 data 
            if len(Class_data[className]) < 100 :
                Class_data[className].append(image.numpy())
        
            
for name in ClassSelected:
    Class_data[name] = np.array(Class_data[name])
    print(f"{name} : {Class_data[name].shape[0]}")

airplane : 100
horse : 100
bird : 100
cat : 100
deer : 100


In [42]:
# just to check if the image is correct for each label

# for class_name in ClassSelected:
#     image = Class_data[class_name][0]
#     image = np.transpose(image, (1, 2, 0))
#     image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
   
#     cv2.imshow(class_name, image)
#     cv2.waitKey(0)

# cv2.destroyAllWindows() 

# Now start model training for testing Decision Tree

In [43]:
# Data is in 3D, it has to be convert to 1D for scikit learn decision tree
X = []
y = []

classint = {name: i for i, name in enumerate(ClassSelected)}

for name in ClassSelected:
    image = Class_data[name]
    for imgdata in image:
        img_flat = imgdata.flatten()
        X.append(img_flat)
        y.append(classint[name])

In [44]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [45]:
X = np.array(X)
y = np.array(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=100)
modelDecisionTree = DecisionTreeClassifier(random_state=100)
modelDecisionTree.fit(X_train, y_train)

DecisionTreeClassifier(random_state=100)

In [46]:
modelDecisionTree.score(X_test, y_test)

0.33